In [11]:
from IPython.display import display, Markdown
import copy
def latexify(x):
    out = '$' + x + '$'
    return out

def lprint(x):
    display(Markdown(latexify(latex(x))))

# Implicit Function Theorem

The implicit function theorem is a core ingredient in bifurcation theory, in this notebook we will explore how to apply it algorithmically 

## Statement of theorem

We will consider functions of the form:

$$f : \mathbb{R}^n+m \rightarrow \mathbb{R}^m $$

usually k-times continuously differentiable for $k \in \mathbb{N}$

The implicit function theorem is used to characterise the zero set of the function around a know solution, suppose (without loss of generality) that:

$$f(0) = 0$$

We then split $\mathbb{R}^n$ into the product of two spaces:

$$\mathbb{R}^{n+m} = \mathbb{R}^n \times \mathbb{R}^m$$ 

From here on we denote $\mathbb{R}^n = X$ and $\mathbb{R}^m = Y$

So that:

$$f : X \times Y \rightarrow \mathbb{R}^m$$

If we then suppose that the derivtative with respect to $Y$ is an isomorphism at 0, i.e:

$$\partial_{Y}f(0,0)$$ is invertible

Then the Implict Function Theorem states that there exists, open subsets of $X$ and $Y$ and a k-times differentiable function:

$$h : U_{X} \rightarrow U_{y}$$

That paramatrises the zero set of $f$ close to zero:

$$\{ (x,y) \in U_{X} \times U_{Y} \;| \;\; f(x,y) = 0 \} = \{ (x,h(x)) \;\; | \;\; x \in U_{X} \}$$

## Computing $h$

We will now construct an algorithm, that can produce a Taylor polynomial approximation of the $h$ function, assuming we know that $\mathbb{R}^{n+m}$ meets the conditions required.

Recall that:

$$h(x) = h(a) + \frac{h'(a)}{1!}(x-a) + \frac{h(a)}{2!}(x-a)^2 + \frac{h'(a)}{3!}(x-a)^3+\dotsb = \sum_{k=0}^\infty \frac{h^{\left(k\right)}(a)}{k!} (x-a)^k$$

is the k-th order Taylor expansion of $h$ at 0

We require a way to find the derivatives of $h$, specifically:

$\partial_{X}h$,     $\partial_{XX}h$,     $\partial_{XXX}h$ 

and so on

To do this observe that since $f(x, h(x)) = 0 \;\;\forall x \in U_{X}$

$$ \partial_{X}^{k}f(x,h(x)) = 0 \;\; \forall k \in \mathbb{N}, \forall x \in U_{X}$$

The first couple of applications of this idea are:

$$ 0 = f(x, h(x))$$

$$ 0 = f_{X} + f_{Y}h_{X}$$ 

$$ 0 = f_{XX} + 2f_{XY}h_{X} + f_{YY}h_{X}^2 + f_{Y}h_{XX}$$

(now omitting inner variables for brevity) 

https://math.stackexchange.com/questions/2037753/implicit-function-theorem-second-derivative-calculation-help

By assumption $f_{Y}$ is always invertible, so we can solve for the values of $h_{X}$ and $h_{XX}$.

Note that:

$$ h(0) = 0$$ 

since it can't be anything else, otherwise $(0,0)$, wouldn't be in the zero set - a contradiction

### Iterating the derivative

First we consider how to generate equation of the form above, we can split each term at each plus sign into a generalised term of the form:

$$ c \; f_{\alpha X \beta Y} \prod_{i = 1}^{n} {h_{\gamma _{i} X}^{k_{i}} } $$

I.e. a constant multiplied by a mixed partial of $f$ multiplied by a product of varying powers of varying $X$ partials of h

If we can form an encoding scheme for terms like this, and a way to differentiate them with respect to $X$ then we have a method to generate arbitrarlily high order equations to solve.

#### Differentiating

We consider:

$$ \partial_{X} \left[ c \; f_{\alpha X \beta Y} \prod_{i = 1}^{n} {h_{\gamma_{i} X}^{k_{i}} } \right] $$


$$ = c \left[ \left( \partial_{X} f_{\alpha X \beta Y} \right)\prod_{i = 1}^{n} {h_{\gamma_{i} X}^{k_{i}} } + f_{\alpha X \beta Y}\left( \partial_{X} \prod_{i = 1}^{n} {h_{\gamma_{i} X}^{k_{i}} } \right) \right] $$

Splitting this into the two expressions either side of the $+$

$$ \left( \partial_{X} f_{\alpha X \beta Y} \right)\prod_{i = 1}^{n} {h_{\gamma_{i} X}^{k_{i}} } = \left(f_{(\alpha +1)X \beta Y} + f_{\alpha X (\beta + 1)Y} h_{X} \right) \prod_{i = 1}^{n} {h_{\gamma_{i} X}^{k_{i}}} $$

And:

$$ f_{\alpha X \beta Y}\left( \partial_{X} \prod_{i = 1}^{n} {h_{\gamma_{i} X}^{k_{i}} } \right) = f_{\alpha X \beta Y} \sum_{i = 1}^{n} {\left( \partial_{X} h_{\gamma_{i} X}^{k_{i}} \right) \prod_{\substack{j = 1 \\ j \neq i}}^{k} h_{\gamma _{j}x}^{k_{j}} } = f_{\alpha X \beta Y} \sum_{i = 1}^{n} {\left( k_{i} \; h_{\gamma _{i} X}^{k_{i}-1} \; h_{(\gamma _{i} + 1)X} \right) \prod_{\substack{j = 1 \\ j \neq i}}^{k} h_{\gamma _{j}x}^{k_{j}} } $$

While very complicated, we see that all the terms we computed stay as sums of expressions of the form:

$$ c \; f_{\alpha X \beta Y} \prod_{i = 1}^{n} {h_{\gamma _{i} X}^{k_{i}} } $$

So inductively we see how the code can proceed

#### Coding the expression block storage + representation

Now we build the framework to do this, to stay in the spirit of sage, a latex output will be avaible. For expressions of the form:

$$ c \; f_{\alpha X \beta Y} \prod_{i = 1}^{n} {h_{\gamma _{i} X}^{k_{i}} } $$

Also implemented is the differentiation rule derived earlier, returning a list of ExpressionBlock objects

In [87]:
class ExpressionBlock:
    def __init__(self, c = 1, alpha = 0, beta = 0, ktuples = (), func1 = 'f', func2 = 'h', X = 'X', Y = 'Y'):
        self.c = c
        self.alpha = alpha
        self.beta = beta
        self.ktuples = ktuples # for python reason is important to be immutable, i.e. tuple not list
        self.func1 = func1
        self.func2 = func2
        self.X = X
        self.Y = Y
        
    def __str__(self):
        # overide representation so can be printed as latex
        if self.c == 0:
            return ""
        
        out = ""
        # add the constant
        if self.c != 1:
            out += str(self.c) + " "
            
        # add f
        out += self.func1
        
        # add partials if there are any
        if (self.alpha != 0) or (self.beta != 0):
            out += "_{"
            # just concatenate them
            # TODO use numbers for values greater than 5
            if self.alpha != 0:
                out += self.X*self.alpha + " "
            if self.beta != 0:
                out += self.Y*self.beta + " "
            out += "} "
            
        for pair in self.ktuples:
            # pairs are (gamma, k)
            if pair[1] == 0:
                # zeroth power of anything is 1, factor of 1 changes nothing - skip
                continue
            out += self.func2
            if pair[0] != 0:
                # gamma
                out += "_{" + self.X*pair[0] + "}"
            if pair[1] != 1:
                # k
                # already ruled out possibility of zero
                out += "^{" + str(pair[1]) + "} " 
                
        # and we are done, here just list out the h terms               
        return out

    def _latex_(self):
        # so works with lprint
        return str(self)
    
    def diff(self):
        # apply the rule we worked out before
        # returns a list of ExpressionBlock instances
        
        out = []
        # first half
        temp = copy.deepcopy(self)
        temp.alpha +=1
        out.append(temp)
        
        temp = copy.deepcopy(self)
        temp.beta += 1
        temp.ktuples = ((1,1),) + temp.ktuples # comma so reads as ( (1,1) ) not just (1,1)
        out.append(temp)
        
        # second half
        # loop over each term in the sum
        for idi, ktuple in enumerate(self.ktuples):
            # ktuple = (gamma, k)
            # indexing starts from 0
            temp = copy.deepcopy(self)
            temp.c = temp.c * ktuple[1] # update constant
            
            # remove the term we differentiate here, i=j 
            temp.ktuples = temp.ktuples[:idi] + temp.ktuples[idi+1:]
            
            # add higher X partial
            temp.ktuples = ((ktuple[0] + 1, 1),) + temp.ktuples
            
            # if we end up with h^0 then don't need to add, since will just be a multiple of 1
            if ktuple[1] != 1:
                temp.ktuples = ((ktuple[0], ktuple[1] - 1),) + temp.ktuples

            out.append(temp)
            
        # and we are done
        return out
            
            
            
            

        
        
        

        
        
        
k1 = ExpressionBlock(c = 4,alpha = 2, beta = 1, ktuples = ((3,4),(0,1)))
lprint(k1)
k2 = ExpressionBlock()
lprint(k2)

[lprint(i) for i in k2.diff()]

$ 4 f_{XX Y } h_{XXX}^{4} h $

$ f $

$ f_{X } $

$ f_{Y } h_{X} $

[None, None]

We see that at least for $f$ the rule works, lets wrap this up into a new class that deals with multiple blocks

In [118]:
class Expression:
    def __init__(self, blocks = [ExpressionBlock()]):
        if blocks is None:
            self.blocks = [] # python reasons - list is mutable
        else:
            self.blocks = blocks
            
    def __str__(self):
        if not self.blocks:
            # empty list
            return ""
        out = ""
        for block in self.blocks:
            # should be an instance of the ExpressionBlock class
            out += str(block) + " + "
        
        out = out[:-3] # remove last plus
        
        return out
    
    def _latex_(self):
        # so works with lprint
        return str(self)
    
    
    def diff(self):
        # returns a new Expression object that is the partial x derivative of the old one
        out = []
        for block in self.blocks:
            block_list = block.diff()
            
            # append to the list
            out += block_list
            
        return Expression(blocks = out)
            
        
            
    

e = Expression(blocks = [k1])
lprint(e)
lprint(e.diff())

$ 4 f_{XX Y } h_{XXX}^{4} h $

$ 4 f_{XXX Y } h_{XXX}^{4} h + 4 f_{XX YY } h_{X}h_{XXX}^{4} h + 16 f_{XX Y } h_{XXX}^{3} h_{XXXX}h + 4 f_{XX Y } h_{X}h_{XXX}^{4} $

This all seems to be working fine

In [119]:
b = ExpressionBlock()
lprint(b)

$ f $

Now put into an expresssion so we can differentiate:

In [120]:
e = Expression(blocks = [b])
lprint(e)

$ f $

In [121]:
lprint(e.diff())

$ f_{X }  + f_{Y } h_{X} $

In [122]:
lprint(e.diff().diff())

$ f_{XX }  + f_{X Y } h_{X} + f_{X Y } h_{X} + f_{YY } h_{X}h_{X} + f_{Y } h_{XX} $

We see that this is working as we expect, though w

In [123]:
lprint(e.diff().diff().diff())

$ f_{XXX }  + f_{XX Y } h_{X} + f_{XX Y } h_{X} + f_{X YY } h_{X}h_{X} + f_{X Y } h_{XX} + f_{XX Y } h_{X} + f_{X YY } h_{X}h_{X} + f_{X Y } h_{XX} + f_{X YY } h_{X}h_{X} + f_{YYY } h_{X}h_{X}h_{X} + f_{YY } h_{XX}h_{X} + f_{YY } h_{XX}h_{X} + f_{X Y } h_{XX} + f_{YY } h_{X}h_{XX} + f_{Y } h_{XXX} $

In [124]:
lprint(e.diff().diff().diff().diff())

$ f_{XXXX }  + f_{XXX Y } h_{X} + f_{XXX Y } h_{X} + f_{XX YY } h_{X}h_{X} + f_{XX Y } h_{XX} + f_{XXX Y } h_{X} + f_{XX YY } h_{X}h_{X} + f_{XX Y } h_{XX} + f_{XX YY } h_{X}h_{X} + f_{X YYY } h_{X}h_{X}h_{X} + f_{X YY } h_{XX}h_{X} + f_{X YY } h_{XX}h_{X} + f_{XX Y } h_{XX} + f_{X YY } h_{X}h_{XX} + f_{X Y } h_{XXX} + f_{XXX Y } h_{X} + f_{XX YY } h_{X}h_{X} + f_{XX Y } h_{XX} + f_{XX YY } h_{X}h_{X} + f_{X YYY } h_{X}h_{X}h_{X} + f_{X YY } h_{XX}h_{X} + f_{X YY } h_{XX}h_{X} + f_{XX Y } h_{XX} + f_{X YY } h_{X}h_{XX} + f_{X Y } h_{XXX} + f_{XX YY } h_{X}h_{X} + f_{X YYY } h_{X}h_{X}h_{X} + f_{X YY } h_{XX}h_{X} + f_{X YY } h_{XX}h_{X} + f_{X YYY } h_{X}h_{X}h_{X} + f_{YYYY } h_{X}h_{X}h_{X}h_{X} + f_{YYY } h_{XX}h_{X}h_{X} + f_{YYY } h_{XX}h_{X}h_{X} + f_{YYY } h_{XX}h_{X}h_{X} + f_{X YY } h_{XX}h_{X} + f_{YYY } h_{X}h_{XX}h_{X} + f_{YY } h_{XXX}h_{X} + f_{YY } h_{XX}h_{XX} + f_{X YY } h_{XX}h_{X} + f_{YYY } h_{X}h_{XX}h_{X} + f_{YY } h_{XXX}h_{X} + f_{YY } h_{XX}h_{XX} + f_{XX Y } h_{XX} + f_{X YY } h_{X}h_{XX} + f_{X Y } h_{XXX} + f_{X YY } h_{X}h_{XX} + f_{YYY } h_{X}h_{X}h_{XX} + f_{YY } h_{XX}h_{XX} + f_{YY } h_{XXX}h_{X} + f_{X Y } h_{XXX} + f_{YY } h_{X}h_{XXX} + f_{Y } h_{XXXX} $

### How to evaluate?

This is a good exercise, but how can we use this to get the Taylor polynomial of f?, for each differentiation level, observe each term can be evaluated at a constant number of vectors $v \in X$

In the last huge expansion, four such vectors can evaluated, e.g.:
(without loss of generality to ordering)

$$f_{XYY}h_{XX}h_{X} (v_{1}, v_{2}, v_{3}, v_{4}) = f_{XYY}(v_{1}, h_{XX}(v_{2}, v_{3}), h_{x}(v_{4})) $$

Recalling that we can view derivatives as linear maps

But why didn't we have to worry about the ordering? 

Just consider the case where the $v_{i}$'s are basis vectors:

see that if we evaluate all the but the last term fully, we will have an equation:

$$f_{Y}h_{XXXX}(e_{i_{1}}, e_{i_{2}}, e_{i_{3}}, e_{i_{4}}) = -w$$

Where $w \in Y$ and is the evaluation of the other terms summed

Then using the fact that $f_{Y}$ is invertible:

$$ h_{XXXX}(e_{i_{1}}, e_{i_{2}}, e_{i_{3}}, e_{i_{4}}) = f_{Y}^{-1}(-w)$$

We see that this value can be viewed as a value in the tensor representation of $h_{XXXX}$ as a 4-linear map, one property of these derivative maps is symmetry so we see that for any permutation $\sigma \in S_{4}$

$$ h_{XXXX}(e_{i_{\sigma(1)}}, e_{i_{\sigma(2)}}, e_{i_{\sigma(3)}}, e_{i_{\sigma(4)}}) = f_{Y}^{-1}(-w) $$

But starting from this permuation we could have got an equation (i.e evaluating all the terms with the input vectors in the new ordering):

$$ h_{XXXX}(e_{i_{\sigma(1)}}, e_{i_{\sigma(2)}}, e_{i_{\sigma(3)}}, e_{i_{\sigma(4)}}) = f_{Y}^{-1}(-\tilde{w}) $$

In practice we only care about these cumulative evaluations, and can observe that:

$$w = \tilde{w}$$

by bijectivity of $f_{y}$, and so can plug in the basis vectors in any order we wish, the evaluation will always stay the same.

### Extracting the Taylor polynomial

We want to get a polynomial from the k-th partial of $h$, to do this we evaluate on variants of the basis vector to get coefficients of the term in the polynomial, for example; if we want to find the coefficient of:

$$x_{1}^2 x_{3}$$

then we would evaluate:


$$h_{XXX}(e_{1}, e_{1}, e_{3})$$

Well almost, in fact we would need to combine the coefficients of:

$$x_{1} x_{1} x_{3}, \;\; x_{1} x_{3} x_{1}, \;\; x_{3} x_{1} x_{1}$$

And combine them

See that the formula for the number of these evaluations is:

$$\frac{3!}{2! 1!}$$

Since we can view as a permuation of $S_n$ where some are indistinguishable since variable have the same name

In general:

$$n_{\text{evals}}\left(\prod_{i = 1}^{n} {x_{i}^{k_{i}}}\right) = \frac{n!}{k_{1}! \cdots k_{n}!} $$

But by symmetry of the derivative multilinear map we only need to evaluate once, then multiple by this $n_{\text{evals}}$ value and we are done

#### Tidying up

Now we know that we can swap without worry, lets write a class to simplify these expressions:

In [148]:
# TODO - fix this

class SimpleExpression(Expression):
    def __init__(self, blocks = [ExpressionBlock()]):
        super().__init__(blocks)

    def diff(self):
        return SimpleExpression(super().diff().blocks)
        
    def simplify(self):

        # first standardise each ExpressionBlock
        for block in self.blocks:
            if not block.ktuples:
                # no h's
                continue
            # first update the h's to be largest to smallest
            # ktuple = (gamma, k) thankfully in right order
            block.ktuples = sorted(block.ktuples, reverse = True)

            # collects like terms, i.e. gamma is the same
            max_gamma = block.ktuples[0][0] # since sorted
            temp_tuples = ()
            for i in range(max_gamma,-1,-1):
                like_terms = [term for term in block.ktuples if term[0] == i]
                if not like_terms:
                    # no terms of this form
                    continue
                    
                total = 0
                for term in like_terms:
                    total += term[1] # sum up the powers since a^x * a^y = a^(x+y)
                    
                temp_tuples += ((i, total), )
                
            block.ktuples = temp_tuples
            
    


        
a = SimpleExpression().diff().diff()
simplify(a)
lprint(a)

lprint(simplify(a.diff()))


        
        

$ f_{XX }  + f_{X Y } h_{X} + f_{X Y } h_{X} + f_{YY } h_{X}^{2}  + f_{Y } h_{XX} $

$ \mathrm{None} $

In [132]:
for i in range(3,-1,-1):
    print(i)

3
2
1
0


In [149]:
# TODO will need to use tensors
# since h will lead to arbitrary elements of Y
# numpy implementation probably the best way to go
# but will need a way to extract partial information in a sensible way to build the tensor